In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import rasterio
import matplotlib.pyplot as plt

met_df = pd.read_csv('kelp_data/metadata.csv')
not_kelp = met_df[~(met_df['type'] == 'kelp')]

In [ ]:
traindf = pd.read_csv('kelp_data/train.csv')
testdf = pd.read_csv('kelp_data/test.csv')

In [ ]:
cloud_band = np.fromfile('allstacked-band7.npy',dtype=np.int16)
band_1 = np.fromfile('allstacked-band1.npy',dtype=np.int16)
no_cloud_b1 = band_1[~cloud_band.astype(bool)].flatten()
band_1 = band_1[band_1 > 0]

In [ ]:
_ = plt.hist(no_cloud_b1,bins=100, log=True)

In [ ]:
_ = plt.hist(band_1,bins=500, log=True)

In [ ]:
_ = plt.hist(no_cloud_b1,bins=500, log=True)

In [ ]:
band_1.max()

In [ ]:
_ = plt.hist(no_cloud_b1[no_cloud_b1>=0], bins=500)

In [ ]:


for i in range(6,7):
    agg = []
    for _, r in tqdm(not_kelp.iterrows()):
        name = r['filename']
        if r['in_train']:
            img = rasterio.open('kelp_data/train_satellite/'+name)
        else:
            img = rasterio.open('kelp_data/test_satellite/'+name)
        agg.append(img.read(i))
    stacked = np.stack(agg,axis=2)
    np.save(file='allstacked-band'+str(i)+'.npy',arr=stacked)
    plt.hist(stacked[~(stacked==-32_768)].flatten(),bins=100, log=True)
    plt.title("Histogram of Band "+str(i))
    plt.show()


### Key Observations:
- Logarithmic scaling along the y axis makes the histograms more readable
    - why is this the case for this dataset?
- perhaps it is more sensible to give the model log-scaled data?
- presence of abnormally large counts of data at the maximum
- how do I effectively clean the data for use in the model? -> segment the cloud bands
- how do I effectively use the cloud band and elevation data? -> segment the land pixels